# Stochastic Block Model

#### Comparison of Kosaraju’s Algorithm and Tarjan’s Algorithm

In [1]:
import networkx as nx
import time

# ----------- Generate Directed SBM Graph -----------
def generate_directed_sbm_graph():
    sizes = [50, 50, 50]
    probs = [
        [0.8, 0.05, 0.02],
        [0.05, 0.7, 0.03],
        [0.02, 0.03, 0.6]
    ]
    undirected_G = nx.stochastic_block_model(sizes, probs, seed=42)
    directed_G = nx.DiGraph()
    directed_G.add_edges_from(undirected_G.edges())
    return directed_G

# ----------- Kosaraju’s Algorithm (Iterative) -----------
def kosaraju_scc_iterative(graph):
    def dfs_order_iterative(graph):
        visited = set()
        finish_stack = []
        for node in graph.nodes():
            if node in visited:
                continue
            stack = [(node, False)]
            while stack:
                current, expanded = stack.pop()
                if expanded:
                    finish_stack.append(current)
                elif current not in visited:
                    visited.add(current)
                    stack.append((current, True))
                    for neighbor in graph.neighbors(current):
                        if neighbor not in visited:
                            stack.append((neighbor, False))
        return finish_stack

    def dfs_collect_scc(graph, nodes_in_order):
        visited = set()
        sccs = []
        for node in reversed(nodes_in_order):
            if node in visited:
                continue
            stack = [node]
            scc = []
            while stack:
                current = stack.pop()
                if current in visited:
                    continue
                visited.add(current)
                scc.append(current)
                for neighbor in graph.neighbors(current):
                    if neighbor not in visited:
                        stack.append(neighbor)
            sccs.append(scc)
        return sccs

    finish_order = dfs_order_iterative(graph)
    transposed = graph.reverse(copy=True)
    return dfs_collect_scc(transposed, finish_order)

# ----------- Tarjan’s Algorithm (Iterative) -----------
def tarjans_scc_iterative(graph):
    index_counter = [0]
    index = {}
    lowlink = {}
    on_stack = set()
    visited = set()
    sccs = []
    stack = []
    to_visit = []

    for v in graph.nodes():
        if v in visited:
            continue

        to_visit.append((v, 'visit'))

        while to_visit:
            node, action = to_visit.pop()

            if action == 'visit':
                if node in visited:
                    continue
                visited.add(node)
                index[node] = index_counter[0]
                lowlink[node] = index_counter[0]
                index_counter[0] += 1
                stack.append(node)
                on_stack.add(node)
                to_visit.append((node, 'postvisit'))
                for neighbor in reversed(list(graph.successors(node))):
                    if neighbor not in visited:
                        to_visit.append((neighbor, 'visit'))
                    elif neighbor in on_stack:
                        lowlink[node] = min(lowlink[node], index[neighbor])

            elif action == 'postvisit':
                for neighbor in graph.successors(node):
                    if neighbor in on_stack:
                        lowlink[node] = min(lowlink[node], lowlink[neighbor])

                if lowlink[node] == index[node]:
                    scc = []
                    while True:
                        w = stack.pop()
                        if w in on_stack:
                            on_stack.remove(w)
                        scc.append(w)
                        if w == node:
                            break
                    sccs.append(scc)

    return sccs

# ----------- Benchmark Function -----------
def benchmark_algorithm(name, func, graph, runs=10):
    total_time = 0
    result_sccs = None
    for i in range(runs):
        g_copy = graph.copy()
        start = time.time()
        sccs = func(g_copy)
        end = time.time()
        total_time += (end - start)
        if i == 0:
            result_sccs = sccs
    avg_time = total_time / runs
    return avg_time, len(result_sccs)

# ----------- Main Execution -----------
if __name__ == "__main__":
    G = generate_directed_sbm_graph()

    print("Benchmarking on Directed SBM Graph...")
    kos_time, kos_count = benchmark_algorithm("Kosaraju", kosaraju_scc_iterative, G)
    tarj_time, tarj_count = benchmark_algorithm("Tarjan", tarjans_scc_iterative, G)

    print("\n--- SCC Benchmark Results (SBM, 10 runs) ---")
    print(f"Kosaraju Time: {kos_time:.4f} sec — SCCs: {kos_count}")
    print(f"Tarjan   Time: {tarj_time:.4f} sec — SCCs: {tarj_count}")


Benchmarking on Directed SBM Graph...

--- SCC Benchmark Results (SBM, 10 runs) ---
Kosaraju Time: 0.0072 sec — SCCs: 150
Tarjan   Time: 0.0006 sec — SCCs: 150


#### Running Stochastic Block Model using FW BW trim parallel algorithm

In [2]:
import networkx as nx
from concurrent.futures import ThreadPoolExecutor
from collections import deque
import time

# ----------- Generate Directed SBM Graph -----------
def generate_directed_sbm_graph():
    sizes = [50, 50, 50]
    probs = [
        [0.8, 0.05, 0.02],
        [0.05, 0.7, 0.03],
        [0.02, 0.03, 0.6]
    ]
    undirected_G = nx.stochastic_block_model(sizes, probs, seed=42)
    directed_G = nx.DiGraph()
    directed_G.add_edges_from(undirected_G.edges())
    return directed_G

# ----------- Forward and Backward BFS -----------
def forward_reach(G, start_node):
    visited = set()
    queue = deque([start_node])
    while queue:
        node = queue.popleft()
        if node not in visited:
            visited.add(node)
            queue.extend(G.successors(node))
    return visited

def backward_reach(G, start_node):
    visited = set()
    queue = deque([start_node])
    while queue:
        node = queue.popleft()
        if node not in visited:
            visited.add(node)
            queue.extend(G.predecessors(node))
    return visited

# ----------- FW-BW Parallel SCC Finder -----------
def find_scc_parallel(G):
    remaining_nodes = set(G.nodes())
    sccs = []

    while remaining_nodes:
        pivot = next(iter(remaining_nodes))

        with ThreadPoolExecutor(max_workers=2) as executor:
            future_fwd = executor.submit(forward_reach, G, pivot)
            future_bwd = executor.submit(backward_reach, G, pivot)
            forward_set = future_fwd.result()
            backward_set = future_bwd.result()

        scc = forward_set & backward_set
        sccs.append(scc)

        remaining_nodes -= scc
        G.remove_nodes_from(scc)

    return sccs

# ----------- Main Execution -----------
if __name__ == "__main__":
    G = generate_directed_sbm_graph()
    print("Benchmarking FW-BW (Parallel) on Directed SBM Graph...")

    total_time = 0
    final_sccs = None

    for i in range(10):
        g_copy = G.copy()
        start = time.time()
        sccs = find_scc_parallel(g_copy)
        end = time.time()
        run_time = end - start
        total_time += run_time
        print(f"Run {i + 1}: {run_time:.4f} seconds")
        if i == 0:
            final_sccs = sccs

    avg_time = total_time / 10
    print(f"\nAverage FW-BW Time: {avg_time:.4f} seconds")
    print(f"SCCs in first run: {len(final_sccs)}")
    print(f"Largest SCC size: {len(max(final_sccs, key=len))}")


Benchmarking FW-BW (Parallel) on Directed SBM Graph...
Run 1: 0.0350 seconds
Run 2: 0.0217 seconds
Run 3: 0.0205 seconds
Run 4: 0.0203 seconds
Run 5: 0.0197 seconds
Run 6: 0.0195 seconds
Run 7: 0.0195 seconds
Run 8: 0.0193 seconds
Run 9: 0.0199 seconds
Run 10: 0.0198 seconds

Average FW-BW Time: 0.0215 seconds
SCCs in first run: 150
Largest SCC size: 1
